# Circle Packing

## Introduction

We aim to pack some circles

In [ ]:
cd("/home/coopar7/Documents/code/BlochTorreyExperiments-master/")

In [ ]:
using Traceur
using BenchmarkTools
using StaticArrays
using JuAFEM
using JuAFEM: vertices, faces, edges
using MATLAB
using LinearMaps
using DifferentialEquations
using Optim
using Cuba
using Distributions

include("Experiments/MyelinWaterOrientation/Geometry/geometry_utils.jl")
include("Experiments/MyelinWaterOrientation/Geometry/circle_packing.jl")
include("Experiments/MyelinWaterOrientation/Utils/mesh_utils.jl")
include("Experiments/MyelinWaterOrientation/Utils/blochtorrey_utils.jl")
Revise.track("Experiments/MyelinWaterOrientation/Geometry/geometry_utils.jl")
Revise.track("Experiments/MyelinWaterOrientation/Geometry/circle_packing.jl")
Revise.track("Experiments/MyelinWaterOrientation/Utils/mesh_utils.jl")
Revise.track("Experiments/MyelinWaterOrientation/Utils/blochtorrey_utils.jl")

In [ ]:
# α or k == R_shape, θ == R_scale
R_mu = 0.46 # Axon mean radius [um] ; this is taken to be outer radius
R_shape = 5.7 # Axon radius shape parameter for Gamma distribution (Xu)
R_scale = R_mu / R_shape # Axon radius scale parameter [um]
R_σ = sqrt(R_shape)*R_scale; # Axon radius variance

In [81]:
revise()

In [82]:
const Dim = 2
Ncircles = 150
rs = rand(Gamma(R_shape, R_scale), Ncircles);
os = initialize_origins(rs);

In [83]:
ϵ = 0.1*R_mu
α = 5e-6

@time circles_opt, opt_result = pack_circles(rs; initial_origins=os, distancescale = R_mu, alpha=α, epsilon=ϵ);

 30.444404 seconds (578.88 k allocations: 82.942 MiB, 0.10% gc time)


In [84]:
opt_result

Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [28.350819435993728,20.28378514270802, ...]
 * Minimizer: [8.944489685040136,3.7791861189254083, ...]
 * Minimum: 7.560200e+00
 * Iterations: 1569
 * Convergence: true
   * |x - x'| ≤ 1.0e-32: false 
     |x - x'| = 1.78e-15 
   * |f(x) - f(x')| ≤ 1.0e-32 |f(x)|: true
     |f(x) - f(x')| = 0.00e+00 |f(x)|
   * |g(x)| ≤ 1.0e-12: false 
     |g(x)| = 2.68e-07 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 1676
 * Gradient Calls: 1570

In [ ]:
ConstrainedOpts = Optim.Options(iterations = 100_000,
                                g_tol = 1e-12,
                                allow_f_increases = false)

@time circles_opt_con, opt_result_con = pack_circles(rs;
                                                     initial_origins = origin.(circles_opt),
                                                     constrained = true,
                                                     epsilon = ϵ,
                                                     Opts = ConstrainedOpts);

In [ ]:
opt_result_con

In [ ]:
cs_plot = circles_opt_con;

In [85]:
cs_plot = circles_opt;

In [86]:
revise()

In [87]:
Nmin = 50; # points for smallest circle
h0 = 2pi*mean(c->radius(c), cs_plot)/Nmin; # approximate scale
#h0 = ϵ
eta = 4.0; # approx ratio between largest/smallest edges
b_box = bounding_box(cs_plot)

Rectangle{2,Float64}([-0.278523, -8.56682], [13.0418, 6.19216])

In [88]:
fullgrid, subgrids = square_mesh_with_circles(b_box, cs_plot, h0, eta, isunion=true);

LoadError: [91mInterruptException:[39m

In [43]:
revise()

In [89]:
#@benchmark estimate_density(cs_plot)

In [98]:
#@time estimate_density(cs_plot)
alphas = 0.05:0.05:1.0;
[alphas estimate_density.([cs_plot], alphas)]

20×2 Array{Float64,2}:
 0.05  0.783334
 0.1   0.851842
 0.15  0.806773
 0.2   0.800488
 0.25  0.79556 
 0.3   0.808523
 0.35  0.804221
 0.4   0.802882
 0.45  0.80533 
 0.5   0.805654
 0.55  0.80165 
 0.6   0.803959
 0.65  0.80498 
 0.7   0.799339
 0.75  0.796968
 0.8   0.794453
 0.85  0.794037
 0.9   0.786342
 0.95  0.765423
 1.0   0.741566

In [92]:
@time estimate_density_monte_carlo(cs_plot; integrator = Cuba_integrator(:cuhre))

(0.7970663221578592, 0.000643579673417094, 0.0)

  1.749681 seconds (6.00 M allocations: 366.263 MiB, 10.38% gc time)


In [47]:
for method in [:suave, :vegas, :divonne, :cuhre]
    res = estimate_density_monte_carlo(cs_plot; integrator = Cuba_integrator(method))
    @show (method, res...)
end

(method, res...) = (:suave, 0.8490709847102297, 0.0006590817226559582, 1.0)
(method, res...) = (:vegas, 0.8012469865743564, 0.0003958422096424144, 5.5024340051677935e-6)
(method, res...) = (:divonne, 0.8009061881441923, 8.040755274328503e-5, 0.21076153715890822)
(method, res...) = (:cuhre, 0.8042865462752294, 0.0016211952575296818, 0.0)


In [37]:
#@benchmark estimate_density_monte_carlo(cs_plot; integrator = Cuba_integrator(:suave))

In [38]:
#@benchmark estimate_density_monte_carlo(cs_plot; integrator = Cuba_integrator(:vegas))

In [39]:
#@benchmark estimate_density_monte_carlo(cs_plot; integrator = Cuba_integrator(:divonne))

In [40]:
#@benchmark estimate_density_monte_carlo(cs_plot; integrator = Cuba_integrator(:cuhre))

In [ ]:
overlap_mat = zeros(Bool, Ncircles, Ncircles);
[overlap_mat[i,j] = is_overlapping(cs_plot[i], cs_plot[j]) for i in 1:Ncircles for j in 1:Ncircles]
overlap_mat;

In [ ]:
dist_mat = zeros(Ncircles, Ncircles);
for i in 1:Ncircles-1, j in 1:i-1
    dist_mat[i,j] = signed_edge_distance(cs_plot[i], cs_plot[j])
end
min_dist = minimum(x->x==zero(x) ? Inf : x, dist_mat)
eps_dist = ϵ
@show min_dist
@show eps_dist
@show 100*(eps_dist - min_dist)/eps_dist;

In [ ]:
dist_mat;

In [ ]:
revise()

In [ ]:
bcircle = crude_bounding_circle(cs_plot)

In [ ]:
fullgrid, subgrids = square_mesh_with_circles(bbox, [circles_opt..., bcircle], h0, eta, isunion=true);

In [ ]:
#x0 = copy(reinterpret(Float64, os))
#origins_chunk = Float64[]
#rs_chunk = Float64[]
#
#@time begin
#    Alg = LBFGS()
#    opts = Optim.Options(iterations=10)
#    fs = (origins, radii) -> energy_sum_overlap_squared_distances(c_0,origins,radii,Val{Dim}) + λ*energy_sum_squared_distances(c_0,origins,radii,Val{Dim})
#    for i = 1:Ncircles-1
#        push!(origins_chunk, x0[2i-1:2i]...)
#        push!(rs_chunk, rs[i])
#
#        result = optimize(origins -> fs(origins, rs_chunk), origins_chunk, Alg, opts) # partial minimization
#        origins_chunk = copy(Optim.minimizer(result))
#
#        @show (i, Optim.minimum(result))
#    end
#    x = copy(origins_chunk)
#    result = optimize(origins -> fs(origins, rs), x, Alg, Optim.Options(iterations=100_000)) # full minimization
#end

In [ ]:
for i in 1:Ncircles-1
    c_i = circles_opt[i]
    for j in i+1:Ncircles
        c_j = circles_opt[j]
        
        #if circle_edge_distance(c_i, c_j) < 0
        #    @show (i, j, c_i, c_j)
        #end
        #@show circle_edge_distance(c_i, c_j)^2
        dx = (origin(c_i)-origin(c_j))
        @show norm(dx)
    end
end

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*